In [16]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML
import pandas as pd

C:\Users\Yuxuan Zheng\AppData\Local\Temp\ipykernel_9132\241899341.py:2: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [17]:
conc = 12

In [18]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
import random

# Constants
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

# P980 = 3*10**4  # our PD

def system(state, t, x, P980, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]

In [19]:
syn = pd.read_csv(f'conc_{conc}.csv')

In [20]:
from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(x, P980, c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total


# x_values = [4, 6, 8, 10, 12, 15, 50]  # Range of C2 values from 2% to 40%


In [21]:
p10_blue = [2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p10_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p12_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p12_blue = [4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p15_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p15_blue = [4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]

power = {10: (p10_nir, p10_blue), 12: (p12_nir, p12_blue), 15: (p15_nir, p15_blue)}

In [22]:
NIR_values = {x: [] for x in power[conc][0]}
blue_total_values = {x: [] for x in power[conc][1]}

count = 0
for index, row in syn.iterrows():
    c1, c2, c3, c4, k31, k41, k51 = row['c1'], row['c2'], row['c3'],row['c4'],row['k31'],row['k41'],row['k51']
    c1, c2, c3, c4, k31, k41, k51 = 10**c1, 10**c2, 10**c3, 10**c4, 10**k31, 10**k41, 10**k51
    for index, x in enumerate(power[conc][0]):
        NIR, _ = compute_values_for_x(conc, x, c1, c2, c3, c4, k31, k41, k51)
        NIR_values[x].append(NIR)
    for index, x in enumerate(power[conc][1]):
        _, blue = compute_values_for_x(conc, x, c1, c2, c3, c4, k31, k41, k51)
        blue_total_values[x].append(blue)
    # print(NIR_values[4])
    # print(blue_total_values[4])
    # count += 1
    # if count >=100:
    #     break


In [23]:
nir_mean_values = [np.mean(NIR_values[x]) for x in power[conc][0]]
nir_std_values = [np.std(NIR_values[x]) for x in power[conc][0]]

blue_mean_values = [np.mean(blue_total_values[x]) for x in power[conc][1]]
blue_std_values = [np.std(blue_total_values[x]) for x in power[conc][1]]

In [24]:
nir_mean_values, nir_std_values

([21.999424006297303,
  52.67889581252566,
  141.36527984070403,
  405.78111588102,
  548.5120710392509,
  835.5442512441937,
  1346.415725390071,
  1923.8840568591331,
  2570.7801781547746,
  3432.9941923811903,
  4110.757107395738,
  4646.70358569736,
  5077.976369816444,
  5535.43327698052,
  5813.597853618158],
 [2.448473399566872,
  5.725822298143263,
  14.603582102836326,
  37.527064044049695,
  48.4500171223644,
  68.80438607641071,
  102.92826564259641,
  139.90512803975716,
  177.92277486575404,
  221.17249214780637,
  251.48098444917082,
  277.6224766303525,
  304.1542428518975,
  342.27893291705954,
  372.5687773648516])

In [25]:
blue_mean_values, blue_std_values

([7.037609503271139,
  12.187523733601278,
  26.752985590922755,
  67.06671364064097,
  135.9335647679737,
  244.1205750498128,
  442.35326507675825,
  643.7860469785587,
  832.6112431980391,
  1003.933722343478,
  1204.7848677204051,
  1336.5929222626721],
 [0.7288930241169218,
  1.1116142522100956,
  1.9610780381296833,
  3.4945108419008877,
  4.874179065771853,
  6.13956005342937,
  11.457026418141558,
  21.525446111456006,
  33.19420944223494,
  44.75395587978623,
  58.885868086727314,
  68.28831721599398])

In [26]:
nir_mean_values + nir_std_values + nir_mean_values[::-1]

[21.999424006297303,
 52.67889581252566,
 141.36527984070403,
 405.78111588102,
 548.5120710392509,
 835.5442512441937,
 1346.415725390071,
 1923.8840568591331,
 2570.7801781547746,
 3432.9941923811903,
 4110.757107395738,
 4646.70358569736,
 5077.976369816444,
 5535.43327698052,
 5813.597853618158,
 2.448473399566872,
 5.725822298143263,
 14.603582102836326,
 37.527064044049695,
 48.4500171223644,
 68.80438607641071,
 102.92826564259641,
 139.90512803975716,
 177.92277486575404,
 221.17249214780637,
 251.48098444917082,
 277.6224766303525,
 304.1542428518975,
 342.27893291705954,
 372.5687773648516,
 5813.597853618158,
 5535.43327698052,
 5077.976369816444,
 4646.70358569736,
 4110.757107395738,
 3432.9941923811903,
 2570.7801781547746,
 1923.8840568591331,
 1346.415725390071,
 835.5442512441937,
 548.5120710392509,
 405.78111588102,
 141.36527984070403,
 52.67889581252566,
 21.999424006297303]

In [27]:
import json
fig = go.Figure()

# Add the main trace (mean curve)
fig.add_trace(go.Scatter(x=power[conc][0], y=nir_mean_values, mode='lines+markers', name='Mean_NIR_Simulation'))
fig.add_trace(go.Scatter(x=power[conc][1], y=blue_mean_values, mode='lines+markers', name='Mean_blue_Simulation'))

# Add the uncertainty shade trace
fig.add_trace(go.Scatter(
    x=power[conc][0] + power[conc][0][::-1],
    y=[i+j for i, j in zip(nir_mean_values,nir_std_values)] + [i-j for i, j in zip(nir_mean_values,nir_std_values)][::-1],
    fill='toself',
    fillcolor='rgba(170, 68, 153, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='NIR_simulation'
))

fig.add_trace(go.Scatter(
    x=power[conc][1] + power[conc][1][::-1],
    y=[i+j for i, j in zip(blue_mean_values,blue_std_values)] + [i-j for i, j in zip(blue_mean_values,blue_std_values)][::-1],
    fill='toself',
    fillcolor='rgba(170, 68, 153, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='NIR_simulation'
))

# exp_blue_total = [4.5*10**3, 5.7*10**3, 5.5*10**3, 3.5*10**3, 2*10**3, 1.2*10**3, 0.6*10**3]
# exp_NIR = [5*10**3, 8.5*10**3, 6.6*10**3, 7.5*10**3, 6*10**3, 5.5*10**3, 0.6*10**3]
# exp_sum = [i+j for i, j in zip(exp_blue_total, exp_NIR)]


# exp_sum = [i+j for i, j in zip(exp_blue_total, exp_NIR)]

# print(int(min(min(blue_mean_values)) / 1e3),
#                             int(max(max(blue_mean_values)) / 1e3) + 1)

# y_ticks = [i for i in range(int(min(blue_mean_values) / 1e3),
#                             int(max(blue_mean_values) / 1e3) + 1)]

# y_ticks = [i for i in range(int(min(min(exp_sum), min(blue_total_values)) / 1e3),
#                             int(max(max(exp_sum), max(blue_total_values)) / 1e3) + 1)]

# fig.update_yaxes(tickvals=[tick * 1e3 for tick in y_ticks],
#                  ticktext=[f"{tick} × 10^3" for tick in y_ticks])


fig.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2000)
fig.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')

# ... Your existing layout and formatting ...

fig.update_layout(
    title_text='NaYbF4: (1-x)% Yb, x% Tm',  # Main Title
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=800, height=600,
    xaxis_title='Power Density (W/cm^2)',
    yaxis_title='Intensity(kpps)',
    title_x=0.42,  # Center the main title
    title_y=0.95,  # Adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
)

with open(f'./synthetic_data/syn_{conc}.json', 'r') as f:
    loaded_json_data = f.read()
loaded_data = json.loads(loaded_json_data)
exp_blue_total = loaded_data.get('blue', [])
exp_NIR = loaded_data.get('NIR', [])
fig.add_trace(go.Scatter(x= power[conc][0], y= exp_NIR, mode='markers', name='NIR_exp', marker=dict(color='rgb(170, 68, 153)', size=10)))
fig.add_trace(go.Scatter(x= power[conc][1], y= exp_blue_total, mode='markers', name='Blue_exp', marker=dict(color='#0099C6', size=10)))

# Show the plot
fig.show()
# print('hihi')

In [28]:
truth = loaded_data.get('param', [])

In [29]:
c1_exps = syn['c1']
c2_exps = syn['c2']
c3_exps = syn['c3']
c4_exps = syn['c4']
k31_exps = syn['k31']
k41_exps = syn['k41']
k51_exps = syn['k51']

In [30]:
fig = go.Figure()

# Adding subplots for each parameter
params = [c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps]
param_names = ["c1_exp", "c2_exp", "c3_exp", "c4_exp", "k31_exp", "k41_exp", "k51_exp"]

for param, name in zip(params, param_names):
    fig.add_trace(go.Box(
        y=param,
        name=name
    ))

    truth_value = truth[param_names.index(name)]  # Assuming the order matches
    fig.add_trace(go.Scatter(
        x=[name],
        y=[truth_value],
        mode='markers',
        marker=dict(size=10, color='red'),
        name='Input Parameters'
    ))

fig.update_xaxes(showgrid=True, gridwidth=2, gridcolor='white')
fig.update_yaxes(showgrid=True, gridwidth=2, gridcolor='white')

fig.update_layout(
        title=f"Parameter exponents of top ten individuals",
        title_font=dict(size=24, family="Courier New, monospace"),
        title_x=0.5,  # center
        title_y=0.9,
        width=640, height=560,
        yaxis_title="Parameter Exponent Value",
        xaxis_title="Seven Parameter Exponents",
        xaxis=dict(
            title_font=dict(size=22, family="Courier New, monospace"),
            tickfont=dict(size=22, family="Courier New, monospace")
        ),
        yaxis=dict(
            title_font=dict(size=22, family="Courier New, monospace"),
            tickfont=dict(size=22, family="Courier New, monospace")
        )
    )


fig.show()